In [1]:
import pandas as pd
import numpy as np

In [2]:
educacion   = pd.read_csv('~/orga-datos/datos_preprocesados/fiuba_1_postulantes_educacion.csv')
genero_edad = pd.read_csv('~/orga-datos/datos_preprocesados/fiuba_2_postulantes_genero_y_edad.csv')
postulantes = pd.read_csv('~/orga-datos/v3/fiuba_3_vistas.csv', usecols=['idpostulante']).append(
    pd.read_csv('~/orga-datos/v3/fiuba_4_postulaciones.csv', usecols=['idpostulante']))

In [3]:
postulantes = postulantes.append(pd.DataFrame(educacion['idpostulante']).append(pd.DataFrame(genero_edad['idpostulante']))).drop_duplicates()

In [4]:
len(postulantes)

519509

In [5]:
educacion.head()

,idpostulante,nombre,estado
0,1d2B,Universitario,En Curso
1,6M3jr,Universitario,En Curso
2,NrlQx,Terciario/Técnico,En Curso
3,Y9RKV,Universitario,En Curso
4,ZL608,Universitario,En Curso


In [6]:
ESCALA_NIVEL_EDUCATIVO = {
    ('Otro', 'Abandonado'): 0, # Jardin de infantes
    ('Otro', 'En Curso'): 0, #
    ('Otro', 'Graduado'): 0, #
    ('Secundario', 'Abandonado'): 1, # Sec incompleto
    ('Secundario', 'En Curso'): 1, # Sec incompleto
    ('Secundario', 'Graduado'): 2, # Sec completo
    ('Terciario/Técnico', 'Abandonado'): 2,
    ('Terciario/Técnico', 'En Curso'): 2,
    ('Terciario/Técnico', 'Graduado'): 2.5,
    ('Universitario', 'Abandonado'): 3,
    ('Universitario', 'En Curso'): 3,
    ('Universitario', 'Graduado'): 4,
    ('Posgrado', 'Abandonado'): 5,
    ('Posgrado', 'En Curso'): 5,
    ('Posgrado', 'Graduado'): 6,
    ('Master', 'Abandonado'): 7,
    ('Master', 'En Curso'): 7,
    ('Master', 'Graduado'): 8,
    ('Doctorado', 'Abandonado'): 9,
    ('Doctorado', 'En Curso'): 9,
    ('Doctorado', 'Graduado'): 10
}
educacion['nivel_educativo'] = educacion.apply(lambda x: ESCALA_NIVEL_EDUCATIVO[(x['nombre'], x['estado'])], axis=1)

In [7]:
educacion = pd.DataFrame(educacion.groupby('idpostulante')['nivel_educativo'].max()).reset_index()

In [8]:
educacion.head()

,idpostulante,nivel_educativo
0,0z5Dmrd,4.0
1,0z5JW1r,3.0
2,0z5VvGv,2.0
3,0zB01pE,3.0
4,0zB026d,2.5


In [9]:
postulantes = pd.merge(educacion, postulantes, on='idpostulante', how='right')

In [10]:
postulantes['nivel_educativo_real'] = ~postulantes['nivel_educativo'].isna()

In [11]:
postulantes.head()

,idpostulante,nivel_educativo,nivel_educativo_real
0,0z5Dmrd,4.0,True
1,0z5JW1r,3.0,True
2,0z5VvGv,2.0,True
3,0zB01pE,3.0,True
4,0zB026d,2.5,True


In [12]:
genero_edad['sexo'] = genero_edad['sexo'].map(lambda x: {'MASC': 1, 'FEM': -1}[x] if x in ('MASC', 'FEM') else 0)

In [13]:
genero_edad['edad'] = genero_edad['fechanacimiento'].map(lambda x: 2018 - int(x.split('-')[0]) if isinstance(x, str) else 0)

In [14]:
genero_edad.head()

,idpostulante,fechanacimiento,sexo,edad
0,eo2p,1981-02-16,1,37
1,1d2B,1976-02-28,1,42
2,EBO0,1973-09-11,-1,45
3,a6MKW,1974-06-07,1,44
4,6MWd4,1974-12-19,1,44


In [15]:
postulantes = pd.merge(postulantes, genero_edad[['idpostulante', 'sexo', 'edad']], on='idpostulante', how='left')

In [16]:
postulantes['edad_real'] = False
postulantes.loc[(postulantes['edad'] > 14) & (postulantes['edad'] < 90),'edad_real'] = True
postulantes['sexo'] = postulantes['sexo'].fillna(0)
postulantes.head()

,idpostulante,nivel_educativo,nivel_educativo_real,sexo,edad,edad_real
0,0z5Dmrd,4.0,True,1.0,53.0,True
1,0z5JW1r,3.0,True,1.0,47.0,True
2,0z5VvGv,2.0,True,1.0,31.0,True
3,0zB01pE,3.0,True,1.0,29.0,True
4,0zB026d,2.5,True,-1.0,40.0,True


In [17]:
postulantes[postulantes['edad'].isna()]

,idpostulante,nivel_educativo,nivel_educativo_real,sexo,edad,edad_real
478456,bNGlW9,NaN,False,0.0,NaN,False
478459,YDeND,NaN,False,0.0,NaN,False
478470,aRjLq,NaN,False,0.0,NaN,False
478475,PdLOlE,NaN,False,0.0,NaN,False
478479,LmlRAq,NaN,False,0.0,NaN,False
478488,zdRRAk,NaN,False,0.0,NaN,False
478489,PZvOa,NaN,False,0.0,NaN,False
478490,Y0mQNY,NaN,False,0.0,NaN,False
478491,oBWDva,NaN,False,0.0,NaN,False
478494,WEY62B,NaN,False,0.0,NaN,False


In [18]:
postulantes.to_csv('~/orga-datos/v3/postulantes.csv', index=False)